## Run a ML pipeline on Vertex Pipelines and Log Runs in Vertex AI Experiments 

## 1 - Pip Install Relevant Libraries

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

!pip3 install {USER_FLAG} --force-reinstall 'google-cloud-aiplatform>=1.15' -q --no-warn-conflicts
!pip3 install {USER_FLAG} kfp -q --no-warn-conflicts

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 2 - Define Constants

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  rewolfe-demo-gcct


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "rewolfe-demo-gcct"  # @param {type:"string"}

In [2]:
#set your region 
REGION = "us-central1"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

In [3]:
#set timestamp to avoid collisions between multiple users

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [4]:
#set cloud storage bucket 
BUCKET_NAME = "experiments-demo-gaming-data"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

## 3 - Create Cloud Storage Bucket & Copy Over Data from Public Bucket

In [5]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://experiments-demo-gaming-data/...


In [5]:
#verify access 
! gsutil ls -al $BUCKET_URI

                                 gs://experiments-demo-gaming-data/model/
                                 gs://experiments-demo-gaming-data/pipelines/
                                 gs://experiments-demo-gaming-data/player_data/


In [7]:
# copy the data over to your cloud storage bucket
DATASET_URI = "gs://cloud-samples-data/vertex-ai/structured_data/player_data" 

!gsutil cp -r $DATASET_URI $BUCKET_URI

Copying gs://cloud-samples-data/vertex-ai/structured_data/player_data/data.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/vertex-ai/structured_data/player_data/labels.csv [Content-Type=text/csv]...
- [2 files][  5.1 MiB/  5.1 MiB]                                                
Operation completed over 2 objects/5.1 MiB.                                      


## 4 - Import Libraries & Define Additional Constants

In [6]:
import logging
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import kfp.v2.compiler as compiler
# Pipeline Experiments
import kfp.v2.dsl as dsl
# Vertex AI
from google.cloud import aiplatform as vertex_ai
from kfp.v2.dsl import Artifact, Input, Metrics, Model, Output, component
from typing import NamedTuple

In [7]:
#Define additional constants


TASK = "regression"
MODEL_TYPE = "tensorflow"
EXPERIMENT_NAME = f"{PROJECT_ID}-{TASK}-{MODEL_TYPE}-{TIMESTAMP}"

# Pipeline
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/player_data/data.csv"
LABEL_URI = f"{BUCKET_URI}/player_data/labels.csv"
MODEL_URI = f"{BUCKET_URI}/model"
DISPLAY_NAME = "experiments-demo-gaming-data"
BQ_DATASET = "player_data"
BQ_LOCATION = "US"  
VIEW_NAME = 'dataset_test'
PIPELINE_JSON_PKG_PATH = "experiments_demo_gaming_data.json"
PIPELINE_ROOT = f"gs://{BUCKET_URI}/pipeline_root"

## 5 - Initialize Vertex AI SDK & Setup our Training Job as a Pipeline Component

In [8]:
#initialize vertex AI SDK 
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [9]:
@component(
    packages_to_install=[
        "numpy==1.21.0",
        "pandas==1.3.5", 
        "scikit-learn==1.0.2",
        "tensorflow==2.9.0",
    ]
)
def custom_trainer(
    train_uri: str,
    label_uri: str,
    dropout_rate: float,
    learning_rate: float,
    epochs: int,
    model_uri: str,
    metrics: Output[Metrics], 
    model_metadata: Output[Model], 
    

):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Dropout
    from tensorflow.keras.metrics import Metric 
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_absolute_error
    import numpy as np
    from math import sqrt
    import os
    import tempfile

    # set variables and use gcsfuse to update prefixes
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    def get_logger():

        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, 
        label_path: str
    ) -> (pd.DataFrame): 
        
        
        #load data into pandas dataframe
        data_0 = pd.read_csv(train_path)
        labels_0 = pd.read_csv(label_path)
        
        #drop unnecessary leading columns
        
        data = data_0.drop('Unnamed: 0', axis=1)
        labels = labels_0.drop('Unnamed: 0', axis=1)
        
        #save as numpy array for reshaping of data 
        
        labels = labels.values
        data = data.values
    
        # Split the data
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=7)
    
        #Convert data back to pandas dataframe for scaling
        
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
        train_labels = pd.DataFrame(train_labels)
        test_labels = pd.DataFrame(test_labels)
        
        #Scale and normalize the training dataset
        
        scaler = StandardScaler()
        scaler.fit(train_data)
        train_data = pd.DataFrame(scaler.transform(train_data), index=train_data.index, columns=train_data.columns)
        test_data = pd.DataFrame(scaler.transform(test_data), index=test_data.index, columns=test_data.columns)
        
        return train_data,train_labels, test_data, test_labels 
    
        """ Train your Keras model passing in the training data and values for learning rate, dropout rate,and the number of epochs """

    def train_model(
        learning_rate: float, 
        dropout_rate: float,
        epochs: float,
        train_data: pd.DataFrame,
        train_labels: pd.DataFrame):
 
        # Train tensorflow model
        param = {"learning_rate": learning_rate, "dropout_rate": dropout_rate, "epochs": epochs}
        model = Sequential()
        model.add(Dense(500, input_dim=train_data.shape[1], activation= "relu"))
        model.add(Dropout(param['dropout_rate']))
        model.add(Dense(100, activation= "relu"))
        model.add(Dense(50, activation= "relu"))
        model.add(Dense(1))
            
        model.compile(
        tf.keras.optimizers.Adam(learning_rate= param['learning_rate']),
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError()])
        
        model.fit(train_data, train_labels, epochs= param['epochs'])
        
        return model  

    # Evaluate predictions with MAE
    def evaluate_model_mae(model, test_data, test_labels):
        
        dtest = pd.DataFrame(test_data)
        pred = model.predict(dtest)
        mae = mean_absolute_error(test_labels, pred)
        return mae
    
    # Evaluate predictions with RMSE
    def evaluate_model_rmse(model, test_data, test_labels):

        dtest = pd.DataFrame(test_data)
        pred = model.predict(dtest)
        rmse = np.sqrt(np.mean((test_labels - pred)**2))
        return rmse    
 
    
    #Save your trained model in GCS     
    def save_model(model, model_path):

        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}"        
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save(model_path + '/model_tensorflow')

        
    # Main ----------------------------------------------
    
    train_data, train_labels, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(learning_rate, dropout_rate, epochs, train_data,train_labels )
    mae = evaluate_model_mae(model, test_data, test_labels)
    rmse = evaluate_model_rmse(model, test_data, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------

    #convert numpy array to pandas series
    mae = pd.Series(mae)
    rmse = pd.Series(rmse)

    #log metrics and model artifacts with ML Metadata. Save metrics as a list. 
    metrics.log_metric("mae", mae.to_list()) 
    metrics.log_metric("rmse", rmse.to_list()) 
    model_metadata.uri = model_uri


## 6 - Build our Pipeline

In [10]:
#define our workflow

@dsl.pipeline(name="gaming-custom-training-pipeline")
def pipeline(
    train_uri: str,
    label_uri: str,
    dropout_rate: float,
    learning_rate: float,
    epochs: int,
    model_uri: str,
):

    custom_trainer(
        train_uri,label_uri, dropout_rate,learning_rate,epochs, model_uri
    )

In [12]:
#compile our pipeline
compiler.Compiler().compile(pipeline_func=pipeline, package_path="gaming_pipeline.json")

## 7 - Define Values for Hyperparameters and Submit our Pipeline Job

In [13]:
runs = [
    {"dropout_rate": 0.001, "learning_rate": 0.001, "epochs": 20},
    {"dropout_rate": 0.002, "learning_rate": 0.002,"epochs": 25},
    {"dropout_rate": 0.003, "learning_rate": 0.003,"epochs": 30},
    {"dropout_rate": 0.004, "learning_rate": 0.004,"epochs": 35},
    {"dropout_rate": 0.005, "learning_rate": 0.005,"epochs": 40},
]

In [14]:
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path="gaming_pipeline.json",
        pipeline_root=PIPELINE_URI,
        parameter_values={
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            **run,
        },
    )
    job.submit(experiment=EXPERIMENT_NAME)

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/718915599574/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20221014191932


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/718915599574/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20221014191932


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/718915599574/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20221014191946')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/gaming-custom-training-pipeline-20221014191946?project=718915599574


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/gaming-custom-training-pipeline-20221014191946?project=718915599574


Associating projects/718915599574/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20221014191946 to Experiment: rewolfe-demo-gcct-regression-tensorflow-20221014191838


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/718915599574/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20221014191946 to Experiment: rewolfe-demo-gcct-regression-tensorflow-20221014191838


## 8 - Leverage the Vertex AI SDK to View Experiments

In [15]:
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.epochs,param.learning_rate,param.label_uri,param.model_uri,param.dropout_rate,param.train_uri,metric.rmse,metric.mae
0,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191946,system.PipelineRun,COMPLETE,40,0.005,gs://experiments-demo-gaming-data/player_data/...,gs://experiments-demo-gaming-data/model,0.005,gs://experiments-demo-gaming-data/player_data/...,[1.6312643984004305],[1.2874774364342425]
1,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191943,system.PipelineRun,COMPLETE,35,0.004,gs://experiments-demo-gaming-data/player_data/...,gs://experiments-demo-gaming-data/model,0.004,gs://experiments-demo-gaming-data/player_data/...,[1.046658621211493],[0.7743505631745374]
2,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191939,system.PipelineRun,COMPLETE,30,0.003,gs://experiments-demo-gaming-data/player_data/...,gs://experiments-demo-gaming-data/model,0.003,gs://experiments-demo-gaming-data/player_data/...,[1.3063697165182702],[1.0374431736275478]
3,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191936,system.PipelineRun,COMPLETE,25,0.002,gs://experiments-demo-gaming-data/player_data/...,gs://experiments-demo-gaming-data/model,0.002,gs://experiments-demo-gaming-data/player_data/...,[1.3626497104081132],[1.0251322637578595]
4,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191932,system.PipelineRun,COMPLETE,20,0.001,gs://experiments-demo-gaming-data/player_data/...,gs://experiments-demo-gaming-data/model,0.001,gs://experiments-demo-gaming-data/player_data/...,[1.260985572628981],[0.9678655664238616]


In [16]:
#check on current status
while True:
    pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
    if all(
        pipeline_state != "COMPLETE" for pipeline_state in pipeline_experiments_df.state
    ):
        print("Pipeline runs are still running...")
        if any(
            pipeline_state == "FAILED"
            for pipeline_state in pipeline_experiments_df.state
        ):
            print("At least one Pipeline run failed")
            break
    else:
        print("Pipeline experiment runs have completed")
        break
    time.sleep(60)

Pipeline experiment runs have completed


In [17]:
# Call the pipeline runs based on the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[0])
print(job.resource_name)
print(job._dashboard_uri())

projects/718915599574/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20221014191946
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/gaming-custom-training-pipeline-20221014191946?project=718915599574


In [18]:
# wait 60 seconds and view state again
import time
time.sleep(60)
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.label_uri,param.dropout_rate,param.learning_rate,param.model_uri,param.epochs,param.train_uri,metric.mae,metric.rmse
0,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191946,system.PipelineRun,COMPLETE,gs://experiments-demo-gaming-data/player_data/...,0.005,0.005,gs://experiments-demo-gaming-data/model,40,gs://experiments-demo-gaming-data/player_data/...,[1.2874774364342425],[1.6312643984004305]
1,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191943,system.PipelineRun,COMPLETE,gs://experiments-demo-gaming-data/player_data/...,0.004,0.004,gs://experiments-demo-gaming-data/model,35,gs://experiments-demo-gaming-data/player_data/...,[0.7743505631745374],[1.046658621211493]
2,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191939,system.PipelineRun,COMPLETE,gs://experiments-demo-gaming-data/player_data/...,0.003,0.003,gs://experiments-demo-gaming-data/model,30,gs://experiments-demo-gaming-data/player_data/...,[1.0374431736275478],[1.3063697165182702]
3,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191936,system.PipelineRun,COMPLETE,gs://experiments-demo-gaming-data/player_data/...,0.002,0.002,gs://experiments-demo-gaming-data/model,25,gs://experiments-demo-gaming-data/player_data/...,[1.0251322637578595],[1.3626497104081132]
4,rewolfe-demo-gcct-regression-tensorflow-202210...,gaming-custom-training-pipeline-20221014191932,system.PipelineRun,COMPLETE,gs://experiments-demo-gaming-data/player_data/...,0.001,0.001,gs://experiments-demo-gaming-data/model,20,gs://experiments-demo-gaming-data/player_data/...,[0.9678655664238616],[1.260985572628981]
